In [1]:
cd D:/Study/sem8/dataset/dataset_bhargavi/megatron/

D:\Study\sem8\dataset\dataset_bhargavi\megatron


In [7]:
!export MYDIR=$D:/Study/sem8/dataset/dataset_bhargavi/megatron/

'export' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
!git clone https://github.com/huggingface/transformers.git

'git' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import csv

# Path to the input CSV file
input_file_path = 'data.csv'

# Path to the output CSV file for the first 100k lines
output_file_path = 'data_small.csv'

# Number of lines to read
num_lines = 100000

# Open the input CSV file in read mode
with open(input_file_path, 'r', newline='') as input_file:
    # Open the output CSV file in write mode
    with open(output_file_path, 'w', newline='') as output_file:
        # Create CSV reader and writer objects
        csv_reader = csv.reader(input_file)
        csv_writer = csv.writer(output_file)

        # Write the first 100k lines to the output file
        for line_num, line in enumerate(csv_reader):
            csv_writer.writerow(line)
            if line_num + 1 >= num_lines:
                break


In [ ]:
!mkdir -p $MYDIR/nvidia/megatron-bert-uncased-345m

In [18]:
!python -m wget https://api.ngc.nvidia.com/v2/models/nvidia/megatron_bert_345m/versions/v0.1_uncased/zip -o D:/Study/sem8/dataset/dataset_bhargavi/megatron/nvidia/megatron-bert-uncased-345m/checkpoint.zip


Saved under D:/Study/sem8/dataset/dataset_bhargavi/megatron/nvidia/megatron-bert-uncased-345m/checkpoint.zip


In [21]:
!pip install transformers

  Obtaining dependency information for sentencepiece from https://files.pythonhosted.org/packages/a2/f6/587c62fd21fc988555b85351f50bbde43a51524caafd63bc69240ded14fd/sentencepiece-0.2.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   - ------------------------------------- 41.0/991.5 kB 960.0 kB/s eta 0:00:01
   ------- -------------------------------- 194.6/991.5 kB 2.0 MB/s eta 0:00:01
   ---------------- ----------------------- 409.6/991.5 kB 3.2 MB/s eta 0:00:01
   --------------------------- ------------ 675.8/991.5 kB 3.5 MB/s eta 0:00:01
   ---------------------------------- ----- 849.9/991.5 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------  983.0/991.5 kB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 3.5 MB/s eta 0:00:00


In [23]:
!python D:/Study/sem8/dataset/dataset_bhargavi/megatron/transformers-main/transformers-main/src/transformers/models/megatron_bert/convert_megatron_bert_checkpoint.py D:/Study/sem8/dataset/dataset_bhargavi/megatron/nvidia/megatron-bert-uncased-345m/checkpoint.zip

2024-04-26 22:44:57.813514: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 22:44:59.670873: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 22:45:14.788841: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "D:\Study\sem8\dataset\dataset_bhargavi\megatron\

In [ ]:
import os
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, MegatronBertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [ ]:
torch.manual_seed(42)

class LogDataset(Dataset):
    def __init__(self, data, tokenizer, max_length,mode="Train"):
        self.mode = mode
        self.texts = data['logs'].tolist()
        if self.mode!="Test":
            self.labels = data['Label'].astype('category').cat.codes.tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        if self.mode!="Test":
            label = self.labels[index]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        if self.mode!="Test":
            return {
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'labels': torch.tensor(label)
            }
        else:
            return {
                'input_ids': input_ids,
                'attention_mask': attention_mask,
            }

In [4]:
data = pd.read_csv('data_small.csv')
data

,Response,merged_features
0,Normal,2008-11-09 8:35:18 PM 143 blk_-160899968791986...
1,Normal,2008-11-09 8:35:18 PM 35 blk_-1608999687919862...
2,Normal,2008-11-09 8:35:19 PM 143 blk_-160899968791986...
3,Normal,2008-11-09 8:35:19 PM 145 blk_-160899968791986...
4,Normal,2008-11-09 8:35:19 PM 145 blk_-160899968791986...
...,...,...
99994,Normal,2008-11-09 8:49:01 PM 645 blk_3844474760914334...
99995,Normal,2008-11-09 8:49:01 PM 646 blk_4250700552550942...
99996,Normal,2008-11-09 8:49:01 PM 648 blk_5892345988522203...
99997,Normal,2008-11-09 8:49:01 PM 648 blk_5892345988522203...


In [ ]:
train_data, val_data = train_test_split(data, test_size=0.2)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('nvidia/megatron-bert-uncased-345m')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

directory = os.path.join('/', 'nvidia/megatron-bert-uncased-345m')

model = MegatronBertForSequenceClassification.from_pretrained(directory, num_labels=3)

model.to(device)

Some weights of MegatronBertForSequenceClassification were not initialized from the model checkpoint at /nvidia/megatron-bert-uncased-345m and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MegatronBertForSequenceClassification(
  (bert): MegatronBertModel(
    (embeddings): MegatronBertEmbeddings(
      (word_embeddings): Embedding(30592, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): MegatronBertEncoder(
      (layer): ModuleList(
        (0-23): 24 x MegatronBertLayer(
          (attention): MegatronBertAttention(
            (ln): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (self): MegatronBertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): MegatronBertSelfOutput(
              (dense): Linear(in_features=1024, out_featur

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

epochs = 10
batch_size = 16
max_length = 128

In [ ]:
train_dataset = LogDataset(train_data, tokenizer, max_length)
val_dataset = LogDataset(val_data, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
for batch in train_dataloader:
    print(batch)
    break

{'input_ids': tensor([[  101,  5511, 14526,  ...,     0,     0,     0],
        [  101,  5511, 14526,  ...,     0,     0,     0],
        [  101,  5511, 14526,  ...,     0,     0,     0],
        ...,
        [  101,  5511, 14526,  ...,     0,     0,     0],
        [  101,  5511, 14526,  ...,     0,     0,     0],
        [  101,  5511, 14526,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0])}


In [ ]:
len(train_dataloader),len(val_dataloader)

(28293, 7074)

In [ ]:
model.train()
for epoch in range(epochs):
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'Loss': total_loss / len(train_dataloader)})

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    scheduler.step()

    print(f'Epoch {epoch + 1}/{epochs}: Train Loss = {total_loss / len(train_dataloader)}, Val Loss = {val_loss / len(val_dataloader)}')

KeyboardInterrupt: 